In [1]:
import numpy as np
import polars as pl
import sys

sys.path.append("../")

from src.img_db import ImageDatabase

In [3]:
# img_db = ImageDatabase("../data/images/original", "../data/images/generated")
# img_db.load_files()

Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: bad Huffman code


In [4]:
# img_db.denoise()

In [5]:
# img_db.crop(crop_size=64)

# X_raw, y = img_db.export(log_abs=False)

In [225]:
import joblib

In [227]:
joblib.dump(img_db.get_data()[0], "../data/all_artworks_denoised_crop64_list.lzma", compress=8, protocol=5)

['../data/all_artworks_denoised_crop64_list.lzma']

In [7]:
# pl.from_numpy(
#     X_raw, 
#     schema=[f"c{i}" for i in range(X_raw.shape[1])], 
#     orient="row"
# ).write_parquet("../data/all_artworks_denoised_zstd.parquet", compression="zstd", compression_level=19)

In [8]:
X_raw2 = pl.read_parquet("../data/all_artworks_denoised.parquet").to_numpy()

img_db2 = ImageDatabase("../data/images/original", "../data/images/generated")
img_db2.load_files()
_, y = img_db2.get_data()

Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: bad Huffman code
Corrupt JPEG data: bad Huffman code


In [208]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve, balanced_accuracy_score, f1_score, make_scorer
from sklearn.preprocessing import StandardScaler

In [224]:
y = y.astype(np.int_)
# X = X_raw2
X = np.log(np.abs(X_raw2 + 1))[:, np.r_[:512, -1024:-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234, stratify=y)

In [216]:
# X_train = X_train[:, np.r_[:512, -512:-0]]
# X_test = X_test[:, np.r_[:512, -512:-0]]
# X_train = X_train[:, :1024]
# X_test = X_test[:, :1024]

X_test.shape

(2793, 1535)

In [194]:
# m1 = LogisticRegression(
#     class_weight="balanced",
#     random_state=1234,
#     n_jobs=cpu_count() - 2,
#     solver="saga",
#     max_iter=800
# ).fit(X_train, y_train)
m1 = LogisticRegressionCV(
    penalty="l2",
    cv=StratifiedKFold(n_splits=5, random_state=1234, shuffle=True),
    class_weight="balanced",
    random_state=1234,
    n_jobs=cpu_count() - 2,
    solver="saga",
    max_iter=500
).fit(X_train, y_train)

/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10

In [195]:
pred_proba1 = m1.predict_proba(X_test)[:, 1]
pred1 = (pred_proba1 > 0.5).astype(np.int_)

In [196]:
balanced_accuracy_score(y_test, pred1)

0.6627637867190561

In [197]:
roc_auc_score(y_test, pred_proba1)

0.7241097198231856

In [198]:
import optuna

In [223]:
m2 = SVC(
    class_weight="balanced"
).fit(X_train, y_train)

pred2 = m2.predict(X_test)

print(balanced_accuracy_score(y_test, pred2))
print(f1_score(y_test, pred2))

0.6066784119818771
0.7542740495024239


In [220]:
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=1234)

def obj(trial: optuna.Trial):
    mod = SVC(
        kernel = trial.suggest_categorical("kernel", ["poly", "rbf", "sigmoid"]),
        C = trial.suggest_float('C', 0.1, 3, log=True),
        gamma=trial.suggest_categorical('gamma', ['auto','scale']),
        class_weight="balanced",
        max_iter=1000
    )

    return cross_val_score(mod, X_train, y_train, cv=kfold, n_jobs=cpu_count() - 2, scoring=make_scorer(balanced_accuracy_score)).mean()

In [221]:
study = optuna.create_study(direction="maximize")
study.optimize(obj, n_trials=40, show_progress_bar=True)
print(study.best_trial)

[I 2023-02-23 00:26:04,265] A new study created in memory with name: no-name-1fafb051-b098-4b20-8c24-4806a37e3c81
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Cons

[I 2023-02-23 00:26:46,381] Trial 0 finished with value: 0.6257460254368975 and parameters: {'kernel': 'rbf', 'svc_c': 94.26183112019605, 'gamma': 'scale'}. Best is trial 0 with value: 0.6257460254368975.


/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Cons

...

/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Cons

[I 2023-02-23 00:28:25,482] Trial 3 finished with value: 0.5 and parameters: {'kernel': 'rbf', 'svc_c': 3.105080505640376e-07, 'gamma': 'auto'}. Best is trial 0 with value: 0.6257460254368975.


/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Cons

[I 2023-02-23 00:28:50,352] Trial 4 finished with value: 0.5 and parameters: {'kernel': 'rbf', 'svc_c': 5.622062043067056e-08, 'gamma': 'scale'}. Best is trial 0 with value: 0.6257460254368975.
[W 2023-02-23 00:28:58,964] Trial 5 failed with parameters: {'kernel': 'sigmoid', 'svc_c': 7.962982988012679e-07, 'gamma': 'auto'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_95761/3542004346.py", line 12, in obj
    return cross_val_score(mod, X_train, y_train, cv=kfold, n_jobs=cpu_count() - 2, scoring=make_scorer(balanced_accuracy_score)).mean()
  File "/home/jcierocki/.local/share/virtualenvs/datathon-kul-voodoo-boyz/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 515, in cross_val_score
    cv_results = cross_vali

KeyboardInterrupt: 